# delete all the source and sink edges, keep the circle

# merge under represented path to the major paths

In [1]:
import sys
#sys.path.append("/usr/local/lib/python3.11/site-packages")
import subprocess
#import networkx as nx
import gzip
#import pyfrost
import numpy
from collections import defaultdict
import matplotlib.pyplot as plot
#import CCGG_extension as CCGG
import json
import AGG
import pandas as pd
#import py4cytoscape as p4c
import pysam

# delete SOURCE and SINK Edges


In [2]:
filename = './hprc_anchor_graphical_genome_k31.gfa'
graph = AGG.GraphicalGenome(filename)


In [8]:
edgelist = graph.outgoing['SOURCE'] + graph.incoming['SINK']
len(edgelist), len("".join([graph.edges[edge]['seq'] for edge in edgelist]))


(128, 3188)

In [9]:
for edge in edgelist:
    del graph.edges[edge]

# Identify under-represented edges

In [29]:
def find_all_reads(graph):
    read_sets = set()
    edgelist = graph.edges.keys()
    for item in edgelist:
        readlist = graph.edges[item]['reads']
        for read in readlist:
            read_sets.add(read)
    return read_sets

read_sets = find_all_reads(graph)
read_num = len(read_sets)

In [33]:

nodelist = graph.anchor.keys()
thresholdlist = [0.0001, 0.0005, 0.05]
edgelist = list(graph.edges.keys())
for threshold in thresholdlist:
    under_represented_edges = []
    for edge in edgelist:
        reads = graph.edges[edge]['reads']
        if len(reads) < read_num * threshold:
            under_represented_edges.append(edge)
    print(threshold, len(under_represented_edges), len(under_represented_edges)/float(len(edgelist)))

0.0001 6 0.0005592841163310962
0.0005 2858 0.2664056674123788
0.05 10285 0.9587061894108874


In [52]:
# Randomly select one
threshold = 0.05
under_represented_edges = []
for edge in edgelist:
    reads = graph.edges[edge]['reads']
    if len(reads) < read_num * threshold:
        under_represented_edges.append(edge)
print(threshold, len(under_represented_edges), len(under_represented_edges)/float(len(edgelist)))

0.05 10285 0.9587061894108874


In [53]:
read_num * threshold

596.0

# Merge under-represented edges

In [55]:
for edge in under_represented_edges:
    src = graph.incoming[edge][0]
    dst = graph.outgoing[edge][0]
    break

# get Series Parallele Graph

In [49]:
class Find_all_Path_between_anchors:
    def __init__(self, graph, start, end, read_sets):
        self.subpath = []
        self.initial_set = read_sets
        self.find_path(graph, start, end, [], 0, self.initial_set)
        
    def find_path(self, g, start, end, sofar, depth, readset):
        
        if start == end:
            sofar1 = sofar + [end]
            if len(readset)>0:
                self.subpath.append((sofar1, readset))
            return
        
        # path not supported
        if len(readset) <1:
            return  
        
        # path not circular
        if start == "SINK":
            return
        
        depth1 = depth+ 1
        
        
        for dst in g.outgoing[start]:   
            # consider the read set in the latest 10 intervals
            if dst.startswith("E"):
                readset1 = readset & set(g.edges[dst]['reads'])
            else:
                readset1 = readset
             
            self.find_path(g, dst, end, sofar = sofar + [start], depth = depth1, readset = readset1)
            
def reconstruct_path_seq(graph, path):
    seq = ""
    for item in path:
        if item.startswith('A'):
            seq += graph.anchor[item]['seq']
        elif item.startswith("E"):
            seq += graph.edges[item]['seq']
        else:
            item += ""
    return seq

In [50]:
class Get_Series_Parallel_Graph:
    
    def __init__(self, graph, read_set):
        self.initial_set = read_set
        self.nodelist = self.series_parallel_graph_nodelist(graph)
        print(self.nodelist)
        self.anchor, self.edges, self.outgoing, self.incoming = self.series_parallel_graph(self.nodelist, graph)
    
    def series_parallel_graph_nodelist(self, subgraph):
        start_node = sorted(subgraph.anchor.keys())[0]
        Nodelist = [start_node]

        edgelist = subgraph.outgoing[start_node]
        node_candidate = []
        for edge in edgelist:
            nodelist = subgraph.outgoing[edge]
            node_candidate += nodelist
            if nodelist[0] not in subgraph.anchor:
                    continue
        node_candidate = sorted(node_candidate)
        node = node_candidate[-1]
        Nodelist.append(node) # append the furthest node
        
        node_avoid_set = set()

        while node != start_node:
            edgelist = subgraph.outgoing[node]
            node_candidate = []
            for edge in edgelist:
                nodelist = subgraph.outgoing[edge]
                # exclude deadend
                if "SINK" in nodelist:
                    continue
                if nodelist[0] not in subgraph.anchor:
                    continue
                if nodelist[0] not in subgraph.outgoing:
                    continue
#                 if nodelist[0] in node_avoid_set:
#                     continue
                node_candidate += nodelist
#             if len(node_candidate) < 1:
#                 Nodelist.remove(node)
#                 node_avoid_set.add(node)
#                 node = Nodelist[-1]
#             else:
            node_candidate = sorted(node_candidate)
            node = node_candidate[-1]
            Nodelist.append(node) # append the furthest node  
                
        return Nodelist
    
    def series_parallel_graph(self, Nodelist, subgraph):
        Node_dict = {}
        Edge_dict = {}
        Outgoing_dict = {}
        Incoming_dict = {}
        for i, node in enumerate(Nodelist[:-1]):
            start_node = node
            end_node = Nodelist[i+1]
            Node_dict[start_node] = subgraph.anchor[start_node]

            path = Find_all_Path_between_anchors(subgraph, start_node, end_node, read_sets)
            #print(start_node,end_node, len(path.subpath))
            index = 0
            for p, rs in path.subpath:
                edgename = 'E%05d.%04d' % (int(start_node[1:]), index)
                seq = reconstruct_path_seq(subgraph, p[1:-1])
                Edge_dict[edgename] = {}
                Edge_dict[edgename]['seq'] = seq
                Edge_dict[edgename]['src'] = start_node
                Edge_dict[edgename]['dst'] = end_node

                Outgoing_dict[start_node] = Outgoing_dict.get(start_node, []) + [edgename]
                Outgoing_dict[edgename] = [end_node]

                Incoming_dict[end_node] = Incoming_dict.get(end_node, []) + [edgename]
                Incoming_dict[edgename] = [start_node]
        return Node_dict, Edge_dict, Outgoing_dict, Incoming_dict

In [51]:
series_parallelgraph = Get_Series_Parallel_Graph(graph, read_sets)

KeyboardInterrupt: 